In [10]:
import pandas as pd
import numpy as np
import re
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import hamming_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold

# Milestone 3: Traditional statistical and machine learning methods, due Wednesday, April 19, 2017

Think about how you would address the genre prediction problem with traditional statistical or machine learning methods. This includes everything you learned about modeling in this course before the deep learning part. Implement your ideas and compare different classifiers. Report your results and discuss what challenges you faced and how you overcame them. What works and what does not? If there are parts that do not work as expected, make sure to discuss briefly what you think is the cause and how you would address this if you would have more time and resources. 

You do not necessarily need to use the movie posters for this step, but even without a background in computer vision, there are very simple features you can extract from the posters to help guide a traditional machine learning model. Think about the PCA lecture for example, or how to use clustering to extract color information. In addition to considering the movie posters it would be worthwhile to have a look at the metadata that IMDb provides. 

You could use Spark and the [ML library](https://spark.apache.org/docs/latest/ml-features.html#word2vec) to build your model features from the data. This may be especially beneficial if you use additional data, e.g., in text form.

You also need to think about how you are going to evaluate your classifier. Which metrics or scores will you report to show how good the performance is?

The notebook to submit this week should at least include:

- Detailed description and implementation of two different models
- Description of your performance metrics
- Careful performance evaluations for both models
- Visualizations of the metrics for performance evaluation
- Discussion of the differences between the models, their strengths, weaknesses, etc. 
- Discussion of the performances you achieved, and how you might be able to improve them in the future

#### Preliminary Peer Assessment

It is important to provide positive feedback to people who truly worked hard for the good of the team and to also make suggestions to those you perceived not to be working as effectively on team tasks. We ask you to provide an honest assessment of the contributions of the members of your team, including yourself. The feedback you provide should reflect your judgment of each team member’s:

- Preparation – were they prepared during team meetings?
- Contribution – did they contribute productively to the team discussion and work?
- Respect for others’ ideas – did they encourage others to contribute their ideas?
- Flexibility – were they flexible when disagreements occurred?

Your teammate’s assessment of your contributions and the accuracy of your self-assessment will be considered as part of your overall project score.

Preliminary Peer Assessment: [https://goo.gl/forms/WOYC7pwRCSU0yV3l1](https://goo.gl/forms/WOYC7pwRCSU0yV3l1)

## Questions to answer: 

- **What are we predicting exactly?**

So, we are trying to predict movie genres. However, we have that each movie has multiple genres. This leads to the question of how we can predict multiple classifiers for the same object. This more general question is called a multilabel clasification problem. We will explore some of our specifications for this problem below. 

One of the best and most standard solution to do multilable classification is called "one vs. rest" classifiers. These classifiers create n models for each of the n labels. One of the advantages of this model is its interpretability and, for our cases, its ease. We can easily create a pipeline that then does these predictions for us. For an implementation of one vs. all, look at scikit learn: http://scikit-learn.org/dev/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier

We will likely be using this in our early attempts at classification. 

- **What does it means to be succesful? What is our metric for success?**

*adapted from http://people.oregonstate.edu/~sorowerm/pdf/Qual-Multilabel-Shahed-CompleteVersion.pdf*

Here are a few options for our measure of accuracy:

#### Exact Match Ratio
The exact match ratio only considers a correct answer for our multilabel data if it is exactly correct (e.g. if there are three classes, we only classify this as correct if we correctly identify all three classes.) 

#### Accuracy 
Accuracy is a simple way of "goodness of prediction." It is defined as follows 

$$ \frac{1}{n} \sum_i^{n}  \frac{|Y_i\cap Z_i|}{|Y_i \cup Z_i|}$$

Where $$Y_i\cap Z_i $$ refers to the total number of correctly predicted labels over the total number of labels for that instance. So, if for example we predicted [romance, action]  and the true labels were [romance, comedy, horror], this would receive an accuracy of 1/4 because there was one correct prediction and 4 unique labels. 


#### Hamming Loss 
The final and most common form of error for multilable predictions is hamming loss. Hamming loss takes into account both the prediction error (an incorrect error is predicted) and the missing error (a relevant lable is NOT predicted.) this is defined as follows below 

$$ \text{HammingLoss, HL} = \frac{1}{kn} \sum_{i}^{n} \sum_l^k [l \in  Z_i \wedge l \notin Y_i)  + I(l \notin Z_i \wedge  l \in Y_i)]$$

*For this project, we will use the hamming loss, which is defined above.* There is a convenient function in `sklearn` to calculate hamming loss: `sklearn.metrics.hamming_loss`

- What is our first modeling approach? Why? 

- What is our second modeling approach? Why? 

In [11]:
'''
An example of hamming loss. We have true labels:

[0, 1]
[1, 1]

And predicted labels:

[0, 0]
[0, 0]

Hamming loss is .75
'''
hamming_loss(np.array([[0, 1], [1, 1]]), np.zeros((2, 2)))

0.75

### Data Collection & Cleaning

## Decision for dropping
Here we choose to drop the missing data instead of imputing because it is non numerical and avereraging or finding means does not make sense in this scencario

In [12]:
#train = pd.read_csv("../data/train_full.csv")
train = pd.read_csv("train_full.csv")

train = train.rename(columns={'lead actors': 'actors'})

# drop a rogue column
train.drop("Unnamed: 0", axis = 1, inplace = True)
train = train.dropna(axis=0).copy()
print "Dataframe shape:", train.shape
train.head(1)

Dataframe shape: (7143, 32)


,10402,10749,10751,10752,10769,10770,12,14,16,18,...,actors,movie_id,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,0,0,0,0,0,0,0,0,0,1,...,"[u'Amy Adams', u'Jeremy Renner', u'Forest Whit...",329865,Taking place after alien crafts land around th...,25.66195,/hLudzvGfpi6JlwUnsNhXwKKg4j.jpg,2016-11-10,Arrival,False,6.9,3510


In [40]:
# check for null values
train.isnull().any()

10402           False
10749           False
10751           False
10752           False
10769           False
10770           False
12              False
14              False
16              False
18              False
27              False
28              False
35              False
36              False
37              False
53              False
80              False
878             False
9648            False
99              False
adult           False
director        False
actors          False
movie_id        False
overview        False
popularity      False
poster_path     False
release_date    False
title           False
video           False
vote_average    False
vote_count      False
dtype: bool

# Model 1: Random Forest

Some thoughts:
    * Random forests don't accept strings, so we'll need to vectorize all of the string variables or exclude them entirely. 

In [41]:
train.columns

Index([u'10402', u'10749', u'10751', u'10752', u'10769', u'10770', u'12',
       u'14', u'16', u'18', u'27', u'28', u'35', u'36', u'37', u'53', u'80',
       u'878', u'9648', u'99', u'adult', u'director', u'actors', u'movie_id',
       u'overview', u'popularity', u'poster_path', u'release_date', u'title',
       u'video', u'vote_average', u'vote_count'],
      dtype='object')

In [14]:
# dummify string columns: director
dummies_directors = train.director.astype(str).str.strip('[]').str.get_dummies(', ')

In [15]:
dummies_directors.shape

(7143, 2349)

In [16]:
# number of times directors show in the data more than once
for col in dummies_directors.columns:
    if sum(dummies_directors[col]) > 1:
        print col, sum(dummies_directors[col])

 Yeon Sang-Ho 3
Aaron Blaise 3
Aaron Woodley 2
Adam Green 2
Adam McKay 6
Adam Randall 4
Adam Robitel 2
Adam Shankman 11
Adam Wingard 6
Adrian Lyne 2
Adrian Maben 2
Afonso Poyart 3
Aisling Walsh 2
Akira Kurosawa 7
Akiva Goldsman 3
Akiva Schaffer 2
Alain Chabat 4
Alain Resnais 4
Alan Barillaro 2
Alan J. Pakula 4
Alan Myerson 2
Alan Parker 5
Alan Taylor 7
Alastair Fothergill 4
Albert Hughes 6
Alejandro Amenábar 5
Alejandro González Iñárritu 8
Aleksander Bach 3
Alex Garland 2
Alex Gibney 5
Alex Kurtzman 2
Alex Proyas 10
Alex Turner 2
Alex Winter 2
Alex Zamm 2
Alexander Payne 4
Alexander Witt 2
Alexandre Aja 7
Alexandre Astier 3
Alfonso Cuarón 11
Alfred Hitchcock 22
Alfred Vohrer 2
Amy Heckerling 7
Anand Tucker 2
Anatole Litvak 3
Andrea Arnold 2
Andrei Tarkovsky 6
Andrew Adamson 17
Andrew Davis 4
Andrew Dominik 2
Andrew Knight 2
Andrew Niccol 10
Andrew Stanton 11
Andrew V. McLaglen 8
André Øvredal 2
Andy Fickman 5
Andy Muschietti 2
Andy Tennant 4
Andy de Emmony 2
Ang Lee 11
Angelina Jolie 2

In [17]:
# dummify string columns: lead actors
dummies_actors = train.actors.astype(str).str.strip('[]').str.get_dummies(', ') 

In [18]:
dummies_actors.shape

(7143, 6059)

In [19]:
# number of times directors show in the data more than once
for col in dummies_actors.columns:
    if sum(dummies_actors[col]) > 1:
        print col, sum(dummies_actors[col])

Jr.' 4
u"Auli'i Cravalho" 6
u"Beverly D'Angelo" 2
u"Brian d'Arcy James" 2
u"Catherine O'Hara" 9
u"Chris O'Donnell" 4
u"Colin O'Donoghue" 3
u"Conan O'Brien" 2
u"Daragh O'Malley" 3
u"Donald O'Connor" 2
u"Dylan O'Brien" 6
u"Frances O'Connor" 5
u"J. Pat O'Malley" 4
u"Jack O'Connell" 4
u"James D'Arcy" 7
u"Jason O'Mara" 4
u"Jennifer O'Neill" 2
u"Joanna 'JoJo' Levesque" 2
u"Juan 'Doc Restrepo" 2
u"Jun'ichi Okada" 4
u"Keir O'Donnell" 5
u"Ken'ichir\xf4 Matsuda" 3
u"Lupita Nyong'o" 2
u"Maryam d'Abo" 5
u"Maureen O'Hara" 4
u"Nicholas D'Agosto" 3
u"O'Shea Jackson Jr." 2
u"Paige O'Hara" 8
u"Peter O'Toole" 8
u"Q'orianka Kilcher" 3
u"Vincent D'Onofrio" 6
u'50 Cent' 2
u'A.J. Cook' 2
u'AJ Michalka' 2
u'Aamir Khan' 2
u'Aaron Eckhart' 15
u'Aaron Paul' 11
u'Aaron Poole' 3
u'Aaron Stanford' 2
u'Aaron Taylor-Johnson' 7
u'Aaron Tveit' 2
u'Aasif Mandvi' 3
u'Abbie Cornish' 13
u'Abigail Breslin' 4
u'Abigail Cruttenden' 2
u'Abigail Hargrove' 5
u'Achita Sikamana' 3
u'Adam Baldwin' 4
u'Adam Beach' 4
u'Adam Devine' 

In [20]:
colnames = list(dummies_directors.columns) + list(dummies_actors.columns) + list(train.columns)
combined_df = pd.DataFrame(pd.concat([dummies_directors, dummies_actors, train], axis = 1),
                      columns = colnames)

#combined_df.drop(combined_df.index[0])
combined_df.shape

(7143, 8440)

In [22]:
# drop actors and directors columns
combined_df = combined_df.drop('actors', 1)
combined_df = combined_df.drop('director', 1)

# drop overview and titles
combined_df = combined_df.drop('title', 1)
combined_df = combined_df.drop('overview', 1)
combined_df = combined_df.drop('poster_path', 1)


combined_df.head()

,Hugues Martin,Yeon Sang-Ho,A.J. Edwards,Aaron Blaise,Aaron Woodley,Abbas Kiarostami,Abel Ferrara,Abraham Polonsky,Adam Brooks,Adam Curtis,...,878,9648,99,adult,movie_id,popularity,release_date,video,vote_average,vote_count
0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,False,329865,25.661950,2016-11-10,False,6.9,3510
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,False,210577,13.126754,2014-10-01,False,7.9,4669
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,False,27205,12.110450,2010-07-14,False,8.0,11276
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,False,198663,11.037776,2014-09-10,False,7.0,4125
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,False,273248,9.701425,2015-12-25,False,7.5,3162


In [24]:
string_cols = ["director", "actors", "overview", "title"]

string_matrix = train[string_cols]

In [26]:
string_matrix.head(10)

,director,actors,overview,title
0,Denis Villeneuve,"[u'Amy Adams', u'Jeremy Renner', u'Forest Whit...",Taking place after alien crafts land around th...,Arrival
1,David Fincher,"[u'Ben Affleck', u'Rosamund Pike', u'Carrie Co...",With his wife's disappearance having become th...,Gone Girl
2,Christopher Nolan,"[u'Leonardo DiCaprio', u'Joseph Gordon-Levitt'...","Cobb, a skilled thief who commits corporate es...",Inception
3,Wes Ball,"[u""Dylan O'Brien"", u'Ki Hong Lee', u'Kaya Scod...","Set in a post-apocalyptic world, young Thomas ...",The Maze Runner
4,Quentin Tarantino,"[u'Samuel L. Jackson', u'Kurt Russell', u'Jenn...",Bounty hunters seek shelter from a raging bliz...,The Hateful Eight
5,Brad Bird,"[u'Britt Robertson', u'George Clooney', u'Raff...","Bound by a shared destiny, a bright, optimisti...",Tomorrowland
6,David Fincher,"[u'Brad Pitt', u'Morgan Freeman', u'Gwyneth Pa...",Two homicide detectives are on a desperate hun...,Se7en
7,Jon M. Chu,"[u'Jesse Eisenberg', u'Woody Harrelson', u'Mar...",One year after outwitting the FBI and winning ...,Now You See Me 2
8,David Yates,"[u'Daniel Radcliffe', u'Rupert Grint', u'Emma ...",Returning for his fifth year of study at Hogwa...,Harry Potter and the Order of the Phoenix
9,André Øvredal,"[u'Emile Hirsch', u'Brian Cox', u'Ophelia Lovi...",Father and son coroners who receive a mysterio...,The Autopsy of Jane Doe


In [27]:
# Set up helper cleaner function
def cleaner(cell):
    line = cell.replace('[u', '').replace(']', '').replace(',', '').replace("u'", '').replace("'", '')
    line = re.sub("(^|\W)\d+($|\W)", " ", line)
    return line
string_matrix['actors'] = string_matrix['actors'].apply(cleaner)

/Users/lukeheine/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
# trim trailing and leading spaces
string_matrix = string_matrix.apply(lambda col: col.str.strip())

In [29]:
# returns output in scipi format; we want to get back to panadas
vect = CountVectorizer(ngram_range=(1, 3))
vect_df = sp.hstack(string_matrix.apply(lambda col: vect.fit_transform(col)))

In [58]:
# def _coo_to_sparse_series(A, dense_index=False):
#     """ Convert a scipy.sparse.coo_matrix to a SparseSeries.
#     Use the defaults given in the SparseSeries constructor. """
#     s = pd.Series(A.data, pd.MultiIndex.from_arrays((A.row, A.col)))
#     s = s.sort_index()
#     s = s.to_sparse()  # TODO: specify kind?
#     # ...
#     return s
#_coo_to_sparse_series(vect_df)

In [11]:
labels = train.columns[:17]
features = train.columns[17:]
# X = train[features]
X = train[["popularity", "vote_average", "vote_count"]]

In [12]:
genre_ids_df = pd.read_csv("../data/genre_ids.csv")
genre_ids_df.drop("Unnamed: 0", axis = 1, inplace = True)

Currently, our label matrix has 17 rows, meaning that each row has 17 different labels associated with it. This is a big problem because there are 2^17 different possible combinations for each row, and, unless we have a ton of data, we likely won't see more than 1 or 2 instances of a given row from the label matrix. This will make it difficult for our classifier to learn patterns. 

We should probably combine similar genres to make this prediction task more teneble. 

How should we do this combination?

### Evaluating the Random Forest using KFold CV